In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers

### Let's define our function to generate pseudo-data

In [3]:
def Sfunction(k,Q2):
    return 1/(k**2+Q2)

### Let's define the range of $Q^2$ values

In [5]:
Q2vals = np.array(np.linspace(1,10,10))
print(Q2vals)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


### Define Simpson's rule for numerical integraton

In [7]:
def simpsons_rule(f, a, b, N):
    h = (b - a) / N
    x = np.linspace(a, b, N+1)
    y = f(x)
    return h/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])

### Generate FUU values for the given $Q^2$ values

In [9]:
k_min = 0 
k_max = 1
def FUU(Q2arr):
    Fvals = []
    for Q2 in Q2arr:
        def integrand(k):
            return Sfunction(k, Q2)
        # Perform the integration using Simpson's rule
        integral = simpsons_rule(integrand, k_min, k_max, 100)
        Fvals.append(integral)
    return np.array(Q2arr), np.array(Fvals)

Q2, Fvals = FUU(Q2vals)
# Create DataFrame
df = pd.DataFrame({'Q2': Q2, 'FUU': Fvals})

In [10]:
df

,Q2,FUU
0,1.0,0.785398
1,2.0,0.435210
2,3.0,0.302300
3,4.0,0.231824
4,5.0,0.188069
5,6.0,0.158236
6,7.0,0.136584
7,8.0,0.120150
8,9.0,0.107250
9,10.0,0.096853


### Let's create a DNN model

In [17]:
# def DNN_model():
#     model = tf.keras.Sequential([
#         layers.Dense(64, activation='relu', input_shape=(2,)),  # Adjust input shape to (2,)
#         layers.Dense(64, activation='relu'),
#         layers.Dense(1)  # Output layer
#     ])
#     return model

# def DNN_model():
#     model = tf.keras.Sequential([
#         layers.Dense(128, activation='relu', input_shape=(2,)),
#         layers.Dense(256, activation='relu'),
#         layers.Dense(128, activation='relu'),
#         layers.Dense(64, activation='relu'),
#         layers.Dense(1)  # Output layer
#     ])
#     return model

L1_reg = 0.000001

def DNN_model(hidden_layers=2, width=5, activation='relu'):
    inp = tf.keras.Input(shape=(2,))
    initializer = tf.keras.initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=42)
    x = tf.keras.layers.Dense(width, activation=activation, kernel_initializer=initializer, activity_regularizer=tf.keras.regularizers.L1(L1_reg))(inp)
    x1 = tf.keras.layers.Dense(width, activation=activation, kernel_initializer=initializer, activity_regularizer=tf.keras.regularizers.L1(L1_reg))(x)
    x2 = tf.keras.layers.Dense(width, activation=activation, kernel_initializer=initializer, activity_regularizer=tf.keras.regularizers.L1(L1_reg))(x1)
    nnout = tf.keras.layers.Dense(1, kernel_initializer=initializer)(x2)
    mod = tf.keras.Model(inp, nnout)
    return mod


### Let's define the integration layer

In [18]:
class TrapezoidalIntegrationLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TrapezoidalIntegrationLayer, self).__init__(**kwargs)

    def call(self, inputs):
        Smodel, tempk = inputs
        
        # Assuming 'k' is the second dimension
        delta_k = tempk[:, 1:] - tempk[:, :-1]
        integrand = (Smodel[:, 1:] + Smodel[:, :-1]) / 2.0
        trapz_integrated = tf.reduce_sum(integrand * delta_k, axis=1)
        return trapz_integrated

### Let's define the FUU model

In [19]:
def integralmodel():
    tempk = tf.keras.Input(shape=(1), name='k')
    tempQ2 = tf.keras.Input(shape=(1), name='Q2')
    #tempk = np.linspace(0, 1, 100)
    
    # Create an instance of the DNN model
    inputVals = tf.keras.layers.Concatenate()([tempk, tempQ2])
    dnn_model = DNN_model()(inputVals)
    
    # Apply TrapezoidalIntegrationLayer
    integrated_Smodel = TrapezoidalIntegrationLayer()([dnn_model, tempk])
    int_result = tf.keras.layers.Layer()([integrated_Smodel])
    
    return tf.keras.Model([tempk, tempQ2], int_result)


### Initiate and Compile the model

In [20]:
FUUmodel = integralmodel()
FUUmodel.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')

### Split data into training and validation

In [21]:
X = df['Q2']
y = df['FUU']

# Splitting data manually
split_ratio = 0
split_index = int(len(X) * (1 - split_ratio))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

k_array = np.linspace(0, 1, len(X_train))  # Adjust length to match X_train

### Training

In [22]:
# Train the model
FUUmodel.fit([X_train, k_array.reshape(-1, 1)], y_train, epochs=100, batch_size=32)

Epoch 1/100
1/1 [==============================] - 1s 818ms/step - loss: 0.1066
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.1066
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.1066
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.1066
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1066
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1066
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.1066
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.1066
Epoch 9/100
1/1 [==============================] - 0s 7ms/step - loss: 0.1066
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.1066
Epoch 11/100
1/1 [==============================] - 0s 12ms/step - loss: 0.1066
Epoch 12/100
1/1 [==============================] - 0s 7ms/step - loss: 0.1066
Epoch 13/100
1/1 [==============================] - 0s 3ms